**1. I see you used a BDS test. Walk
me through what that told you about your trading strategy's residuals and why that
insight is valuable beyond just looking at the P&L.**

When one designs a trading strategy, one should try to exploit all patters available, the residuals (the difference between our predictions and the actual returns) should be random, what the BDS tells me that the P&L can't is whether I'm missing patterns that I could use to increase my returns, specifically it tells me of I'm missing nonlinear patterns. The fact that the $p-value$ of the BDS test is high tells me that my residuals are random (independently and identically distributed), it tells me that my model is somehow complete since the difference between my predictions and reality do not exhibit any pattern.

The reason that this is valuable is that I can generalize my model to include 
patterns, or try different transformations to see what kind of pattern one might 
be missing, we performed a $GARCH(1,1)$ model on the residuals to realize that
most of the nonlinear relationship between the pair that we do not capture is 
volatility clustering


**2. We often see correlated pairs
blow up. How did you statistically verify that your chosen pair was more than just
correlated and was actually suitable for a mean-reversion strategy? What's the
biggest pitfall in this process?**

Indeed, correlation spikes during crisis, however one needs to keep in mind that correlation only talks about the short-term linear relationship between the pairs, while co-integration is about a long-term, equilibrium/mean-reversal relationship between the two assets in the pair. 

Since the cointegration is done, in a long timescale compared to that of correlation spikes (years instead of weeks) that in itself tells us we're not merely looking at correlation. On the technical side, the Engle-Granger test was performed, this test checks that the the each of the assets individually, undergoes unpredictable movements (it's non-stationary), for a pair to be cointegrated both time series need to be non-stationary, this was check with the Augmented Dickey-Fuller test (ADF), we then use linear regression to find a linear combination of their prices that is stationary (that is it moves around some value, we call  this behaviour mean reverting), again this was checked with ADF.

The biggest pitfall is what is known as "spurious regression", this happens when we confuse a long-term correlation with cointegration, while in theory the ADF test saves us from this however, the linear regression might be a bit off, or changes in the pair assets, macro or other disruptive events, might break down the relation between the series, so one should always keep checking for cointegration to avoid trading what might be a broken strategy.


**3. Your model identified a
high-volatility regime. What was the practical trigger for switching the strategy off?
How would you convince a skeptical portfolio manager that this isn't just
data-snooping and will work going forward?**

The practical trigger for switching the strategy on and off, is a high probability of being in the high volatility regime  (which we call a stressed regime). This probability was set at 0.9, which we believe it is reasonable, while one might think that lower probabilities would be better, to quote Peter Lynch  "Far more money has been lost by investors preparing for corrections or trying to anticipate corrections than has been lost in corrections themselves" so we think it's worth minimizing false negatives.

In order to convince a skeptical portfolio manager, I would point that:

- The regimes are not just statistical constructs, but that they clearly represent calm and distressed markets, that they should think of it as a systematic threshold for the well known VIX that will signal turmoil.
- Our backtesting shows that the strategy outperforms the alternative, this proves that it adds value beyond mere data-snooping
- It adds proactive risk management that is based on a signal to avoid losses rather than reacting to the losses to minimize them, it is a forward looking risk management tool


**4a. This project involves combining
multiple models. Tell me about a time during this project where one of your models
failed or produced a counterintuitive result. How did you troubleshoot and solve the
problem?**

In any of the models we are using the residuals of the pairs, a problem I stumbled
across was the fact than when I first tested the data, the BDS test indicated that 
there was a nonlinear dependence in virtually all regimes and all datasets 
which was counterintuitive, indicating that models
based on a linear pair strategy like the one we had plan would be missing 
the patterns in the data.  

To troubleshoot this we took a look at the z-values over the different scales 
since they were all large  (around two orders of magnitude bigger than expected)
and positive, whenever this happens and we study a large timeframe we should 
suspect the main pattern that we are not explaining is volatility clustering,
we used a $GARCH(1,1)$ model on the residuals and performed the BDS test there 
to verify this assumption, once we did we saw how the residuals were random 
around most scales with a $95\%$ confidence confirming that the pattern
we miss is volatility clustering.


**4b. This project involves combining
multiple models. Tell me about a time during this project where one of your models
failed or produced a counterintuitive result. How did you troubleshoot and solve the
problem?**

When I was looking for conintegration strategies, I was struggling with the fact that the data we used was not cointegrated (XLE vs. XOP). The residuals were failing the  ADF test as the p values were too high. The problem itself had to do with the way the ADF test is done in the statmodels library, basically there are several types of regression on which one can look for the residuals, the default value was not appropiate in this case, by reading the documentation and changing the default value I was not only able to solve the problem but better understand the data and the reasons the cointegration strategy doesn't do so good here, the reason is that the data is only cointegrated if the data had no drift, but it does have a small downward drift up to march 2020. Understanding this fail allowd me to understand how strong the conintegration is in this case (based on how general the regression is in each case)



**4c. This project involves combining
multiple models. Tell me about a time during this project where one of your models
failed or produced a counterintuitive result. How did you troubleshoot and solve the
problem?**

A problem I ran into was that the parameters for the regime switching strategy (Markov Switching Autoregression) were orders of magnitude apart,a clear sign of an unstable and potentially overfit model, making it unreliable for a live trading strategy., To solve this I imposed reasonable  but not overly restrictive, bounds on the parameters.

By constraining the optimization to a meaningful search space, the model's parameters became far more reasonable and the resulting regime classifications were consistent, interpretable, and economically sound. This process was not an act of data-snooping, but a necessary regularization step to ensure the model was well-posed and the resulting parameters were statistically meaningful.